## Anime recommender using Cosine similarity

In this notebook, an user-item based recommender is built ratings each user assigned an anime.

In addition to calculating how similar (close) to items are like jaccard does, cosine also considers the direction of items by representing them as vectors.

In [ ]:
# importing packages
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## KNN Item based recommender

In [ ]:
# import data
df2 = pd.read_csv('../data/processed/cosine_data.csv')

In [ ]:
# pivot table of uis dataframe
uis_table = df2.pivot_table(index='User_ID', columns='name', values='Feedback')

# average rating by row, i.e each movie
avg_ratings = uis_table.mean(axis=1)

# centering ratings around 0
uis_pivot = uis_table.sub(avg_ratings, axis=0)

In [ ]:
uis_pivot.fillna(0, inplace=True)

In [ ]:
# Transpose data to get anime_pivot
anime_pivot = uis_pivot.T

anime_pivot.head()

In [ ]:
# calculating Anime cosine similarity based on user ratings
similarities = cosine_similarity(anime_pivot)

# dataframe of cosine similarity
cosine_similarity_df = pd.DataFrame(
    similarities, index=anime_pivot.index, columns=anime_pivot.index
)

In [ ]:
# function for cosine recommendation
def get_cosine_recommendations(name):
    return cosine_similarity_df[name].sort_values(ascending=False)[1:11].reset_index().rename({name: "Cosine similarity", "name":"Anime"}, axis=1)

In [ ]:
get_cosine_recommendations('Naruto')

## K-nearest neighbors

KNN is another technique used in creating recommendation systems

In [ ]:
# creating data pivot
data_pivot = df2.pivot_table(index="name",columns="User_ID",values="Feedback").fillna(0)

In [ ]:
# compressing our sparse matrix
data_matrix = csr_matrix(data_pivot.values)

In [ ]:
# fitting the KNN algorithm on the sparse matrix using cosine similarity as the metric for calculating the distances
model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
model_knn.fit(data_matrix)

In [ ]:
# function for retrieving knn recommendations
def get_knn_recommendation(name):
    _, indices = model_knn.kneighbors(data_pivot.loc[name,:].values.reshape(1, -1), n_neighbors = 11)
    rec_list = data_pivot.iloc[indices[0]].index[1:].tolist()
    rec_df = df2[np.isin(df2['name'], rec_list)]
    rec_series = rec_df.groupby('name')['Feedback'].mean().sort_values(ascending=False)
    rec = pd.DataFrame(rec_series).reset_index().rename({'name': 'Anime', 'Feedback': 'Average Feedback'}, axis=1)
    return rec

In [ ]:
get_knn_recommendation('Naruto')